In [14]:
!pip install ImageChops

  ERROR: Could not find a version that satisfies the requirement ImageChops (from versions: none)
ERROR: No matching distribution found for ImageChops
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [44]:
mypath = "D:\Mirror\Pictures\CLEAN"
    
from os import listdir
from os.path import isfile, join 
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) ]

from PIL import Image
import os 
import json
from PIL import Image, ExifTags
from datetime import datetime
import os.path, time
import os
import shutil
import cv2
from PIL import ImageChops

import numpy as np
import cv2
import math, operator

from pathlib import Path

def mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err


def diff(path1,path2):
    original = cv2.imread(path1)
    contrast = cv2.imread(path2)

    original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    contrast = cv2.cvtColor(contrast, cv2.COLOR_BGR2GRAY)
    if original.shape[0] != contrast.shape[0]:
        return 100.0
    return mse(original,contrast)

for filename in onlyfiles:
    
    path_to_file = os.path.join(mypath,filename)
    print(path_to_file)
    year = "_"
    if path_to_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        image_exif = Image.open(path_to_file)._getexif()
        if image_exif:
            # Make a map with tag names
            exif = { ExifTags.TAGS[k]: v for k, v in image_exif.items() if k in ExifTags.TAGS and type(v) is not bytes }
            #print(exif)
            if "DateTimeOriginal" not in exif or exif['DateTimeOriginal']:
                print("DateTimeOriginal not exists",path_to_file)
                date_obj = datetime.fromtimestamp(os.path.getmtime(path_to_file))
            else:
                print("exif",exif['DateTimeOriginal'])
                if len(exif['DateTimeOriginal'])>14:
                    # Grab the date
                    date_obj = datetime.strptime(exif['DateTimeOriginal'], '%Y:%m:%d %H:%M:%S')
                else:
                    # Grab the date
                    date_obj = datetime.strptime(exif['DateTimeOriginal'], '%Y:%m:%d %H:%M')

            print(date_obj,date_obj.year)
            year =date_obj.year 
        else:
            print('Unable to get date from exif for %s' % filename)
            date_obj = datetime.fromtimestamp(os.path.getmtime(path_to_file))
            print(date_obj,date_obj.year)
            year =date_obj.year
            
    else:
        print('Unable to get date from exif for %s' % filename)
        date_obj = datetime.fromtimestamp(os.path.getmtime(path_to_file))
        print(date_obj,date_obj.year)
        year =date_obj.year
    
    directory = os.path.join(mypath,str(year))
    if not os.path.exists(directory):
        os.makedirs(directory)        
    source_file = os.path.join(directory,filename)
    print("destination:",path_to_file)
    print("source:",source_file)
    if not os.path.exists(os.path.join(directory,filename)):
        os.rename(path_to_file, os.path.join(directory,filename))
    else:
        if source_file.lower().endswith(('.mov')) :
            shutil.move(path_to_file, os.path.join(directory,"copy-of-"+filename))
            print("moving with new name: copy-of-",filename)
        else:
            dif = diff(source_file,path_to_file)
            print("difference:",diff(source_file,path_to_file))
            if dif == 0.0:
                print(">> moving...",path_to_file,source_file)
                shutil.move(path_to_file,source_file)
            else:
                print("breakşng...",)
                move_file_name = os.path.join(directory,Path(path_to_file).resolve().stem+"(1)"+Path(source_file).suffix)
                print("source:",path_to_file,"dest:",move_file_name)
                shutil.move(path_to_file, move_file_name)
                

            


        

D:\Mirror\Pictures\CLEAN\DSCN5112.JPG
DateTimeOriginal not exists D:\Mirror\Pictures\CLEAN\DSCN5112.JPG
2012-05-10 16:53:24.878608 2012
destination: D:\Mirror\Pictures\CLEAN\DSCN5112.JPG
source: D:\Mirror\Pictures\CLEAN\2012\DSCN5112.JPG
difference: 0.0
>> moving... D:\Mirror\Pictures\CLEAN\DSCN5112.JPG D:\Mirror\Pictures\CLEAN\2012\DSCN5112.JPG
D:\Mirror\Pictures\CLEAN\IMG_0018.jpg
DateTimeOriginal not exists D:\Mirror\Pictures\CLEAN\IMG_0018.jpg
2015-10-11 17:10:56 2015
destination: D:\Mirror\Pictures\CLEAN\IMG_0018.jpg
source: D:\Mirror\Pictures\CLEAN\2015\IMG_0018.jpg
difference: 0.0
>> moving... D:\Mirror\Pictures\CLEAN\IMG_0018.jpg D:\Mirror\Pictures\CLEAN\2015\IMG_0018.jpg


In [12]:
import json
from PIL import Image, ExifTags
from datetime import datetime

def main(filename):
    image_exif = Image.open(filename)._getexif()
    if image_exif:
        # Make a map with tag names
        exif = { ExifTags.TAGS[k]: v for k, v in image_exif.items() if k in ExifTags.TAGS and type(v) is not bytes }
        print(json.dumps(exif, indent=4))
        # Grab the date
        date_obj = datetime.strptime(exif['DateTimeOriginal'], '%Y:%m:%d %H:%M:%S')
        print(date_obj)
    else:
        print('Unable to get date from exif for %s' % filename)

In [ ]:
!pip install exifread